In [4]:
import polars as pl

from swinno_bioeconomy_directionality.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

2025-04-24 17:05:44.842 | INFO     | swinno_bioeconomy_directionality.config:<module>:7 - PROJ_ROOT path is: /Users/research/projects/swinno_bioeconomy_directionality


In [5]:
id_year_bioeconomy = pl.read_csv(RAW_DATA_DIR / "id_year_bioeconomy.csv")
cleaned_bio = pl.read_csv(RAW_DATA_DIR / "cleaned_bioeconomy_query.csv")

In [6]:
id_year_bioeconomy

sinno_id,year,bioeconomy
i64,i64,i64
6462001,1970,0
6475001,1970,0
6853001,1970,0
6856001,1970,1
6857001,1970,0
…,…,…
16373001,2021,1
16374001,2021,0
16375001,2021,0


In [7]:
cleaned_bio

sinno_id,name,description,year
i64,str,str,i64
10025001,"""Certus-stolen""","""Arbetsstol av ny konstruktion …",1975
10042001,null,"""Anläggning för maskinell separ…",1976
10145001,null,"""Ny metod för att på ett effekt…",1979
10148001,null,"""Station för inmätning av timme…",1979
10153001,null,"""Hörselskydd med inbyggd kommun…",1979
…,…,…,…
9937001,"""Mobeel""","""Programvara som möjliggör lagr…",2005
9971001,null,"""Frässkär ; kallat rillskär. De…",1971
9979001,null,"""Svaveldioxid-skrubber avsedd f…",1970


In [8]:
id_year_bioeconomy.filter(pl.col("sinno_id").is_in(cleaned_bio["sinno_id"])).group_by(
    "bioeconomy"
).agg(pl.col("sinno_id").n_unique())

bioeconomy,sinno_id
i64,u32
1,569
0,84


In [9]:
bv = pl.read_csv(RAW_DATA_DIR / "bioeconomy_visions.csv")

In [10]:
bv

sinno_id,bioeconomy_vision,year
i64,str,i64
5880001,"""Not Bioeconomy""",1975
5881001,"""Bioresource Vision""",1975
5883001,"""Bioresource Vision""",1976
5894001,"""Not Bioeconomy""",1974
5897001,"""Not Bioeconomy""",1979
…,…,…
16155001,"""Not Bioeconomy""",2021
16197001,"""Bioresource Vision""",2021
16243001,"""Not Bioeconomy""",2021


In [11]:
cleaned_bio.join(bv, on="sinno_id", how="full").sort("bioeconomy_vision").group_by(
    "bioeconomy_vision"
).agg(pl.col("sinno_id").n_unique())

bioeconomy_vision,sinno_id
str,u32
"""Biotechnology Vision""",19
null,242
"""Bioecology Vision""",54
"""Bioresource Vision""",257
"""Not Bioeconomy""",147


In [12]:
cv = pl.read_csv(PROCESSED_DATA_DIR / "certain_visions.csv")

In [13]:
cv["sinno_id"].n_unique()

655

In [14]:
bv["sinno_id"].n_unique()

655

In [15]:
cleaned_bio.join(cv, on="sinno_id", how="full").filter(
    pl.col("bioeconomy_vision").is_null()
)

sinno_id,name,description,year,sinno_id_right,bioeconomy_vision,year_right
i64,str,str,i64,i64,str,i64
13398001,"""Bioink""","""3d-skrivare som kan skriva ut …",2016,null,null,null
8447001,null,"""Biologisk rening av förorenat …",1989,null,null,null
16069001,"""Nexum""","""En mjukvaruplattform för index…",2020,null,null,null
6893001,null,"""Jonbytarteknik för att behandl…",1973,null,null,null
11449001,null,"""Metod och utrustning för massf…",2010,null,null,null
…,…,…,…,…,…,…
11028001,null,"""Hus färdiga för montering som …",2008,null,null,null
7387001,"""Kiruna Trädplockare""","""Trädplockare. Kraftig maskin s…",1975,null,null,null
13969001,"""Biodolomer""","""Ett nytt nedbrytbart fossilfri…",2019,null,null,null


In [ ]:
from pathlib import Path


tags_path = RAW_DATA_DIR / "tags/"
final_classification_path = RAW_DATA_DIR / "final_classifications/"

tags_dfs = [
    pl.read_csv(
        p, columns=["sinno_id", "bioeconomy_vision", "innovation_type", "notes"]
    )
    for p in tags_path.glob("*.csv")
]
final_classifications = [
    pl.read_csv(
        p,
        columns=[
            "sinno_id",
            "bioeconomy_vision",
            "innovation_type",
            "article_checked",
            "notes",
        ],
    )
    for p in final_classification_path.glob("*innovations-to-check.csv")
]

final_classification_df = pl.concat(final_classifications)
tags_df = pl.concat(tags_dfs)

final_classification_df

sinno_id,bioeconomy_vision,innovation_type,article_checked,notes
i64,str,str,i64,str
6000001,"""9""","""9""",1,null
6039001,"""9""","""9""",1,null
6341001,"""2""","""9""",1,null
6562001,"""9""","""602""",1,null
6740001,"""9""","""9""",1,null
…,…,…,…,…
16276001,null,null,null,null
16280001,null,null,null,null
16287001,null,null,null,null


In [ ]:
not_in_final = tags_df.filter(
    ~pl.col("sinno_id").is_in(final_classification_df["sinno_id"])
)
not_in_final

sinno_id,innovation_type,bioeconomy_vision,notes
i64,str,str,str
6029001,"""9""","""9""",null
6163001,"""601""","""9""",null
6238001,"""106""","""9""",null
6281001,"""9""","""9""",null
6314001,"""601""","""9""",null
…,…,…,…
10050001,null,"""9""",null
10161001,"""207""","""9""",null
10230001,null,"""9""",null


In [ ]:
merged_uncertain_classifications = pl.concat(
    [final_classification_df, not_in_final], how="diagonal"
)
prefered_duplicates = (
    merged_uncertain_classifications.filter(pl.col("sinno_id").is_duplicated())
    .sort("sinno_id")
    .filter(pl.col("article_checked").is_not_null())
)

uncertain_classifications = pl.concat(
    [
        merged_uncertain_classifications.filter(~pl.col("sinno_id").is_duplicated()),
        prefered_duplicates,
    ]
)

In [ ]:
retagged = pl.read_csv(
    RAW_DATA_DIR / "final_classifications/retagged_bioeconomy_visions.csv"
)
retagged

sinno_id,bioeconomy_vision
i64,i64
5999001,9
6031001,2
6110001,2
6945001,3
6945001,2
…,…
12639001,3
12639001,1
12639001,2


In [ ]:
classifications = uncertain_classifications.filter(
    ~pl.col("sinno_id").is_in(retagged["sinno_id"])
)

fc = pl.concat(
    [
        classifications.select(["sinno_id", "bioeconomy_vision"]),
        retagged.with_columns(pl.col("bioeconomy_vision").cast(pl.String)),
    ],
    how="vertical",
)

In [124]:
fc.filter(pl.col("sinno_id").is_duplicated())

sinno_id,bioeconomy_vision
i64,str
7991001,"""2"""
7991001,null
9529001,null
9529001,null
6945001,"""3"""
…,…
11399001,"""1"""
11399001,"""2"""
12639001,"""3"""


In [ ]:
def split_cols(df: pl.DataFrame, col_to_split: str, sep: str) -> pl.DataFrame:
    """Splits a string column into multiple new columns and removes the original column."""
    split_expr = pl.col(col_to_split).cast(pl.Utf8).str.strip_chars().str.split(sep)

    # Apply the split, assuming we don't know the number of splits in advance
    df = df.with_columns(split_expr.alias("__temp_split"))

    max_splits = df.select(pl.col("__temp_split").list.len().max()).item()

    new_cols = [
        pl.col("__temp_split").list.get(i).alias(f"{col_to_split}_{i}")
        for i in range(max_splits)
    ]

    df = df.with_columns(new_cols).drop([col_to_split, "__temp_split"])
    return df


In [ ]:
def clean_codes(
    df: pl.DataFrame,
    column: str,
    code_digits: int,
    remove_whitespace: bool = True,
    drop_na: bool = True,
) -> pl.DataFrame:
    # Start with optional drop_na
    if drop_na:
        df = df.drop_nulls([column])

    df = df.pipe(replace_letters_codes, column)
    col = pl.col(column).cast(pl.Utf8).str.strip_chars()

    # Replace semicolons with commas
    col = col.str.replace_all(";", ",")

    # Replace standalone 0 and 9 with repeated digits (e.g., "0" -> "0000" if code_digits == 4)
    padded_0 = "0" * code_digits
    padded_9 = "9" * code_digits
    col = col.str.replace_all(r"\b0\b", padded_0)
    col = col.str.replace_all(r"\b9\b", padded_9)

    # Replace improperly formatted code sequences with a comma (this part is a bit ambiguous in original logic)
    # Example: insert commas after codes of fixed length unless it's the last in the string.
    # This step requires assumptions about structure. We simplify and fix double commas in next step.

    # Replace double commas with single commas
    col = col.str.replace_all(",,", ",")

    if remove_whitespace:
        col = col.str.replace_all(r"\s", ",")

    # Apply the cleaned column
    df = df.with_columns(col.alias(column))

    return df


def replace_letters_codes(df: pl.DataFrame, col: str) -> pl.DataFrame:
    col_expr = (
        pl.col(col)
        .cast(pl.Utf8)
        .str.replace_all("A", "1")
        .str.replace_all("B", "2")
        .str.replace_all("C", "3")
    )
    return df.with_columns(col_expr.alias(col))

In [ ]:
classifications.filter(pl.col("sinno_id") == 7393001)

sinno_id,bioeconomy_vision,innovation_type,article_checked,notes
i64,str,str,i64,str
7393001,"""2;3""","""207;213""",1,"""2;3: ""vi vill att man se nya s…"


In [117]:
def split_cols(df: pl.DataFrame, col_to_split: str, sep: str = ",") -> pl.DataFrame:
    """Splits a string column into multiple new columns and removes the original column."""
    return df.with_columns(
        pl.col(col_to_split).str.split(sep).list.to_struct(n_field_strategy="max_width")
    ).unnest(col_to_split)


def melt_table(
    df: pl.DataFrame, index: str, col_start: str, value_name: str
) -> pl.DataFrame:
    """Melts wide-format table into long-format, removing blanks and nulls."""
    # Find all columns starting with the prefix
    value_vars = [c for c in df.columns if c.startswith(col_start)]

    melted = df.unpivot(index=index, on=value_vars, value_name=value_name)

    # Remove empty strings and nulls
    melted = melted.filter(
        (pl.col(value_name).is_not_null()) & (pl.col(value_name) != "")
    )

    return melted.select([index, value_name]).sort(index)

In [ ]:
cc = clean_codes(classifications, "bioeconomy_vision", 1)

tc = cc.with_columns(pl.col("bioeconomy_vision").str.strip_chars().str.split(","))
tc = tc.with_columns(
    pl.col("bioeconomy_vision").list.to_struct(n_field_strategy="max_width")
).unnest("bioeconomy_vision")

melt_table(tc, "sinno_id", "field_", "bv")

/var/folders/l7/mzdsc3cd72z1b29sk4zh_tg80000gq/T/ipykernel_20884/583908087.py:4: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  tc = tc.with_columns(pl.col("bioeconomy_vision").list.to_struct(n_field_strategy="max_width")).unnest("bioeconomy_vision")


sinno_id,bv
i64,str
5880001,"""9"""
5881001,"""2"""
5883001,"""2"""
5894001,"""9"""
5897001,"""9"""
…,…
16266001,"""9"""
16276001,"""2"""
16280001,"""2"""


In [26]:
from polars.testing import assert_frame_equal

In [ ]:
id_year_decline = pl.read_csv(RAW_DATA_DIR / "id_year_decline.csv")

In [31]:
assert_frame_equal(id_year_bioeconomy, id_year_decline, check_dtypes=False)

AssertionError: DataFrames are different (value mismatch for column 'bioeconomy')
[left]:  [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,

In [30]:
id_year_bioeconomy

sinno_id,year,bioeconomy
i64,i64,i64
6462001,1970,0
6475001,1970,0
6853001,1970,0
6856001,1970,1
6857001,1970,0
…,…,…
16373001,2021,1
16374001,2021,0
16375001,2021,0
